圖片來源:https://universe.roboflow.com/roboflow-universe-projects/fall-detection-ca3o8/dataset/4

In [ ]:
import os
import cv2
from ultralytics import YOLO
import csv
conf = 0.9
# Path to the directory containing images
input_dir = 'C:/Users/User/Desktop/Fall Detection/train/images' # 存放image資料夾
output_dir = f'C:/Users/User/Desktop/Fall Detection/detect_{conf}' # 偵測後輸出image 資料夾
output_file = f"keypoints_data_{conf}.csv" # image keypoint csv
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load YOLOv8 model for pose detection
model = YOLO('yolov8n-pose.pt')  # Ensure you have the correct YOLOv8 pose model
def process_image(results):
    r = results[0]
    combined_results = []
    for i in range(len(r.keypoints.xyn)):
        for  j  in range(17):
            x, y = r.keypoints.xyn[i][j].numpy()
            confidence = r.keypoints.conf[i][j].item()  # Convert tensor to float
            combined_results.append((x, y, confidence))
    return combined_results
# Process each JPG image in the input directory
all_keypoints_data = []
for idx, filename in enumerate(os.listdir(input_dir)):
    if filename.lower().endswith('.jpg'):
        input_path = os.path.join(input_dir, filename)
        
        # New filename with renaming convention, e.g., prefix "detected_" + original filename
        new_filename = f'detected_{idx:04d}.jpg'  # e.g., detected_0001.jpg, detected_0002.jpg, etc.
        output_path = os.path.join(output_dir, new_filename)

        # Load the image
        image = cv2.imread(input_path)
        # resized image to 640*640
        # resized_image = cv2.resize(image, (640, 640))
        # Perform pose detection
        results = model(image, conf=conf)

        if len(results[0].boxes) == 1:
            # Draw the results on the image
            annotated_image = results[0].plot()
            # Save the annotated image with the new filename
            cv2.imwrite(output_path, annotated_image)
            print(f"Processed and saved: {output_path}")
            keypoints_data = process_image(results)
            keypoints_data.insert(0, new_filename)
            all_keypoints_data.append(keypoints_data)
            
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the header
    header = ["image_name"] + [f"x{i+1}, y{i+1}, confidence{i+1}" for i in range(len(all_keypoints_data[0]) - 1)]
    writer.writerow(header)
    
    # Write the data
    writer.writerows(all_keypoints_data)

print("Data saved to", output_file)


0: 640x640 (no detections), 341.8ms
Speed: 6.2ms preprocess, 341.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 328.0ms
Speed: 4.0ms preprocess, 328.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 322.3ms
Speed: 5.0ms preprocess, 322.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 328.1ms
Speed: 5.0ms preprocess, 328.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 330.4ms
Speed: 3.0ms preprocess, 330.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 321.2ms
Speed: 4.0ms preprocess, 321.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 326.2ms
Speed: 4.3ms preprocess, 326.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 324.4ms
Speed: 4.1ms prepr